In [29]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib
# This is the path I use
#DRIVER_PATH = '/Users/anand/Desktop/chromedriver'
# Put the path for your ChromeDriver here
DRIVER_PATH = '/Users/Usuario/Desktop/Project-Week-3-Data-Thieves/scraping-glassdoor-selenium/chromedriver.exe'


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    queries = ["banana","boniato","brocolli","chicken","eggs","spinach"]  #change your set of querries here
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,140,wd)
        #images_path = '/Users/anand/Desktop/contri/images'  #enter your desired image path
        images_path = 'C:\\Users\\Usuario\\Desktop\\IH_finalproject\\Project-Week-8-Final-Project_fyuji\\data'
        for i in links:
            persist_image(images_path,query,i)
    wd.quit()

Found: 100 search results. Extracting links from 0:100
Found: 141 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcStGal9R1nnRFDNlovI8FXkb6DDYikLHhxZig&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\698d81c8a4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRoEgSgVhcbmJoLkjmzZFpnpyoHAUSVqzFnQQ&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\f2c1636f83.jpg
SUCCESS - saved https://cdn-prod.medicalnewstoday.com/content/images/hero/271/271157/271157_1100.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\d849e91629.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT3pJNZ14hr8rIdUw0rMJA7LGA-MsYbs1Az3w&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\e789959a7c.jpg
SUCCESS - saved https://encr

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTrRgsveIlBneyICECuPZfXsJtzfX3ub8lBdQ&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\eb2c887c8d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS2FNXEQ5g9XiRpjRwlmqok3cZYcCcfja7-ww&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\4d3929d5a1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT9jVnbLdIkGObZl6rAqHaZc5-ujIsMVjX3sw&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\42e20538dc.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS0PGCfwP03n2Q1UHeHa9p9ghWzhD4qTvHv1w&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\7ca517f595.jpg
SUCCESS - saved https://i.ytimg.com/vi/SzgEk5xPG2E/maxresdefault.jpg - as C:\Users\Usuario\Deskt

SUCCESS - saved https://asset.swarovski.com/images/$size_1450/t_swa103/b_rgb:fafafa,c_scale,dpr_3.0,f_auto,w_500/5453571_png/pendientes-no-regrets-banana--multicolor--ba%C3%B1o-en-tono-oro-swarovski-5453571.png - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\20fc5668a2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTdVXyX5-eDQ5ED7YNPmZLi6vlUpSPNfTvuNA&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\b2587b0c88.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/bananas-isolated-on-white-background-260nw-269712092.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\8fc133f68b.jpg
SUCCESS - saved https://media.gettyimages.com/illustrations/rendering-bananas-with-fake-eyelashes-and-a-couple-backwards-illustration-id1074281368?s=612x612 - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRK2Fg2sbcZzWWPmRzaPJN6EfPpRECtVoLRFw&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\1b86660b93.jpg
SUCCESS - saved https://static4.abc.es/media/sociedad/2018/03/26/platanos-potasio-salud-k3gG-U212262406718N6D-620x400@abc-kkuH--620x349@abc.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\3827f2e882.jpg
SUCCESS - saved https://www.jardineriaon.com/wp-content/uploads/2018/06/banana.jpeg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\400a508430.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQg7CGrc87eDMTgUGnuRDssmoPnmc0a1rZrYA&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\banana\5570d710fd.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTcwGuikFqZ6fUuXZv4YZlGkiCTXKk83c

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQOW_HRyB5dHB_qI1YqXkxDieCscwWQk9m_aw&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\boniato\e3070c1e55.jpg
SUCCESS - saved https://valenciafood.es/wp-content/uploads/2018/05/Pasteles-de-boniato.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\boniato\d3f116e53e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS9EC4TeTsLV5mUM4zswhI_vLdyaqJEm6W5Zw&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\boniato\6880200867.jpg
SUCCESS - saved https://www.thermorecetas.com/wp-content/uploads/2016/11/estofado-de-garbanzos-boniato-y-verduras-thermorecetas.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\boniato\23b212ee2e.jpg
SUCCESS - saved https://fitlicioso.com/wp-content/uploads/2019/02/Bolas-de-sesamo-y-boniato-fit-bu%C3%B1u

...

SUCCESS - saved https://www.thespruceeats.com/thmb/48hZzQGVpRdAAGvo7otpmvO0xA4=/3000x3000/smart/filters:no_upscale()/cloud-eggs-4142081-hero-01-5c4a2a2c46e0fb00017deebf.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\eggs\b06806c7db.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTNdGGOzzT456s63ZVKWmrsAhjo16KrXnehww&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\eggs\6f6c5c5ed9.jpg
SUCCESS - saved https://i0.wp.com/cdn-prod.medicalnewstoday.com/content/images/articles/283/283659/a-basket-of-eggs.jpg?w=1155&h=1541 - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\eggs\cf887dadeb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRRx4uHaTqfZfSvvipppNXhI-G20-SsOzxH_w&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\eggs\0879384e05.jpg
SUCCESS - saved https://encrypted

Found: 141 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQzGZa0rk7XiUZ5V9BLaxBAsA9QiJQzjRSKZQ&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\3ba46c40f1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ6aknY8lZXuiUEXNlyYKOJJdv80svnofvRGw&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\f507304f3f.jpg
SUCCESS - saved https://www.healthyseasonalrecipes.com/wp-content/uploads/2017/05/spinach-salad-sq-020.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\29bac81c98.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQhegZ0vxNuRk0L52V9Eur6OVfCP3o9P8b9kw&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\0fb1978462.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRNpZCnRMw1JB8_J9D1rszSmuICZRV1HjhjAw&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\223614705e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT24MqpHBMEjHQDj0BHEQ2UBKqVWknXWFSMgQ&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\cd45c39c0f.jpg
SUCCESS - saved https://www.jessicagavin.com/wp-content/uploads/2020/01/how-to-cook-spinach-10-1200.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\b3677f8c6c.jpg
SUCCESS - saved https://www.rareseeds.com/media/catalog/product/cache/669b2b24490aa03231e7a5aaf581b083/S/w/Swiss-Chard-Perpetual-Spinach-DSC02553.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\0a68d78601.jpg
SUCCESS - saved https://www.saltandlavender.com/wp-content/uploads/2020/01

SUCCESS - saved https://cdn.mos.cms.futurecdn.net/atyrpYQoxdoTzmEgu8HMWE.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\9a8a709042.jpg
SUCCESS - saved https://www.culinaryhill.com/wp-content/uploads/2019/07/Strawberry-Spinach-Salad-Culinary-Hill-HRsquare-07.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\540b528f20.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRNtLKYjA6XU9StKzBd3T1NwAHmRYECjVSyQQ&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\dfe438abe6.jpg
SUCCESS - saved https://unsophisticook.com/wp-content/uploads/2017/05/How-To-Freeze-Fresh-Spinach-720x540.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\35820d2979.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSVVuTWY-wCNpvLPrXBuSNGOO63Dqa1qmpP-w&usqp=CAU - as C:\Users\

SUCCESS - saved https://assets.epicurious.com/photos/5900e0b6fb98ea70e40f7dbe/1:1/w_1332,h_1332,c_limit/Using-the-Root-End-of-Spinach-Greek-Dish-hero-26042017.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\4163716261.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSr4jgrcl9SC0po31BfgUy181KMRJJo9zHOkg&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\3f373bf691.jpg
SUCCESS - saved https://static01.nyt.com/images/2015/10/16/dining/16COOKING-SAUTEDSPINACH2/16COOKING-SAUTEDSPINACH2-articleLarge-v2.jpg - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\ef897fe60c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSED4Aeeek6FzlKSvE73ydvic5WruZkIumw4Q&usqp=CAU - as C:\Users\Usuario\Desktop\IH_finalproject\Project-Week-8-Final-Project_fyuji\data\spinach\2801d049e3.jpg
SUCCESS - saved https://encry

In [13]:
pip install google_images_download

  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14555 sha256=2fc2ffd46f29f1b23073b9afe86fb31d91971cbcb63f1e9313a7c04e5a76ac2a
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\e3\98\42\0d3a76d46cd5a6659afb2f5612d4908ca42d34060973d46727
Successfully built google-images-download
Note: you may need to restart the kernel to use updated packages.


In [14]:
from google_images_download import google_images_download


In [24]:
def downloadimages(query):
	response = google_images_download.googleimagesdownload() 
	# keywords is the search query 
	# format is the image file format 
	# limit is the number of images to be downloaded 
	# print urs is to print the image file url 
	# size is the image size which can 
	# be specified manually ("large, medium, icon") 
	# aspect ratio denotes the height width ratio 
	# of images to download. ("tall, square, wide, panoramic") 
	arguments = {"keywords": query, 
				#"format": "jpg", 
				"limit":4,
				"print_urls":True}
				#"size": "large"}
				#"aspect_ratio": "panoramic" }
	try: 
		response.download(arguments) 
	
	# Handling File NotFound Error	 
	except FileNotFoundError: 
		arguments = {"keywords": query, 
					"format": "jpg", 
					"limit":4, 
					"print_urls":True, 
					"size": "medium"} 
					
		# Providing arguments for the searched query 
		try: 
			# Downloading the photos based 
			# on the given arguments 
			response.download(arguments) 
		except: 
			pass


In [27]:
downloadimages("chicken")


Item no.: 1 --> Item name = chicken
Evaluating...
Starting Download...


Unfortunately all 100 could not be downloaded because some images were not downloadable. 0 is all we got for this search filter!

Errors: 0

